In [49]:
from utils.load_data import load_pt_results, load_behavioral_data, load_pupil_results
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, false_discovery_control

In [42]:
pt_params = load_pt_results().query("model == 'model_2' & param_name != 'mu' & param_name != 'loss'")
behavioral_data = load_behavioral_data(min_RT = 0.2)
pupil_data = load_pupil_results().query("sub != 4")
sub_cond = behavioral_data[["sub", "condition"]].drop_duplicates()
pupil_data = pupil_data.merge(sub_cond, on = ["sub"])
pupil_data = pupil_data.merge(pt_params, on = ["sub", "condition"])

In [58]:
# 3 plots
p_values = []
r_values = []
# for condition in pupil_data["condition"].unique():
for name in pupil_data["name"].unique():
    for param in pupil_data["param_name"].unique():
        d = pupil_data.query("name == @name & param_name == @param")
        r, p = pearsonr(d["estimate"], d["param_value"])
        p_values.append(p)
        r_values.append(r)

p_values = list(false_discovery_control(p_values, method = "by"))

In [59]:
# for condition in pupil_data["condition"].unique():
for name in pupil_data["name"].unique():
    for param in pupil_data["param_name"].unique():
        d = pupil_data.query("name == @name & condition == @condition & param_name == @param")
        pval, rval = p_values.pop(0), r_values.pop(0)
        if pval > 0.05:
            continue
        else:
            print(name, param)
            print(pval, rval)

In [45]:
pupil_data.query("name == @name")["estimate"]

0      0.954407
1      0.954407
6      0.359519
7      0.359519
12     0.417615
         ...   
307    0.672108
312    0.031488
313    0.031488
318    0.388723
319    0.388723
Name: estimate, Length: 108, dtype: float64

In [46]:
pupil_data.query("param_name == @param")["param_value"]

0      0.989402
2      0.989402
4      0.989402
6      1.598506
8      1.598506
         ...   
314    1.113116
316    1.113116
318    1.322579
320    1.322579
322    1.322579
Name: param_value, Length: 162, dtype: float64

In [47]:
pupil_data.query("name == ")

,Unnamed: 0_x,sub,name,estimate,condition,Unnamed: 0_y,param_name,param_value,model
0,0,3,dist_from_indiff,0.954407,equalIndifference,440,lambda,0.989402,model_2
1,0,3,dist_from_indiff,0.954407,equalIndifference,441,rho,0.907968,model_2
2,1,3,gain,-0.086289,equalIndifference,440,lambda,0.989402,model_2
3,1,3,gain,-0.086289,equalIndifference,441,rho,0.907968,model_2
4,2,3,loss,0.051143,equalIndifference,440,lambda,0.989402,model_2
...,...,...,...,...,...,...,...,...,...
319,162,124,dist_from_indiff,0.388723,equalRange,861,rho,1.575457,model_2
320,163,124,gain,0.212436,equalRange,860,lambda,1.322579,model_2
321,163,124,gain,0.212436,equalRange,861,rho,1.575457,model_2
322,164,124,loss,-0.056869,equalRange,860,lambda,1.322579,model_2
